In [109]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [127]:
generator_llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite')
evaluator_llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',max_tokens=1000)
optimizer_llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite')

In [128]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")


In [112]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [113]:
# state
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [ ]:
def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    # send generator_llm
    response = generator_llm.invoke(messages).content

    # return response
    return {'tweet': response, 'tweet_history': [response]}

In [ ]:
# If your first joke is being accepted you can add 
# always reject if {state['iteration']} is 1
# so you can test it for multiple iterations
def evaluate_tweet(state: TweetState):

    # prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [116]:
def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [117]:
def route_evaluation(state: TweetState):

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [123]:
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()



In [124]:
initial_state = {
    "topic": "indian rails",
    "iteration": 1,
    "max_iteration": 5
}
result = workflow.invoke(initial_state)

In [126]:
result

{'topic': 'indian rails',
 'tweet': 'Okay, here\'s a tweet that *tries* to be funny about Indian Railways, keeping in mind I\'m a beginner and aiming for that slightly-off, potentially-cringey humor:\n\n"Just rode an Indian Railways train. Pretty sure I saw a family unpack a full picnic *and* start a game of cricket before we even left the station. #IndianRailways #TravelDiaries #NeverADullMoment ...Also, pretty sure I\'m now fluent in \'head wobble\'."',
 'evaluation': 'approved',
 'feedback': "This tweet successfully captures the chaotic charm of Indian Railways with relatable and humorous observations. The exaggeration of a family starting a cricket game and a full picnic before departure is genuinely funny and well-executed, avoiding the pitfalls of a stale setup-punchline format. The addition of the 'head wobble' line provides a clever and universally understood cultural reference, enhancing the humor without being a generic throwaway. While the core themes of Indian Railways expe

In [95]:
for tweet in result['tweet_history']:
    print(tweet)

Just saw a train in India. Pretty sure it's powered by sheer willpower and the collective hopes of a billion people. Also, snacks. Lots and lots of snacks. #IndianRailways #TravelHumor #DesiLife
